Critical Radii: Semidetached Systems
============================

Setup
-----------------------------

Let's first make sure we have the latest version of PHOEBE 2.3 installed (uncomment this line if running in an online notebook session such as colab).

In [1]:
#!pip install -I "phoebe>=2.3,<2.4"

As always, let's do imports and initialize a logger and a new Bundle.

In [2]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger()

b = phoebe.default_binary()

Semi-Detached Systems
-----------------------------

Semi-detached systems are implemented by constraining the value of requiv to be the same as requiv_max by appyling the 'semidetached' constraint on the 'primary' component.


In [3]:
b.add_constraint('semidetached', 'primary')

<ParameterSet: 1 parameters>

We can view the constraint on requiv by accessing the constraint:

In [4]:
b['requiv@constraint@primary']

<ConstraintParameter: {requiv@primary@component} = 1.000000 * {requiv_max@primary@component} (solar units) => 2.013275176537638 solRad>

Now whenever any of the relevant parameters (q, ecc, syncpar, sma) are changed, the value of requiv will change to match the critical value as defined by requiv_max.

In [5]:
b['requiv_max@constraint@primary']

<ConstraintParameter: {requiv_max@primary@component} = requiv_L1({q@binary@component}, {syncpar@primary@component}, {ecc@binary@component}, {sma@binary@component}, {incl@primary@component}, {long_an@primary@component}, {incl@binary@component}, {long_an@binary@component}, 1) (solar units) => 2.013275176537638 solRad>

Note that the point region in semi-detached systems often falls outside of the atmosphere tables.  Since this requires the full mesh, this cannot be checked in advance with [b.run_checks](../api/phoebe.frontend.bundle.Bundle.run_checks.md) but will raise an error during [b.run_compute](../api/phoebe.frontend.bundle.Bundle.run_compute.md).

In [6]:
b.add_dataset('lc', compute_phases=phoebe.linspace(0,1,101))

<ParameterSet: 78 parameters | contexts: compute, dataset, figure, constraint>

In [8]:
try:
    b.run_compute()
except Exception as err:
    print(err)

Fri, 20 Nov 2020 09:53 UNIVERSE     WARNING Atmosphere parameters out of bounds: Teff=[348.51161081], logg=[-11.66455964], abun=[0.]


Could not compute ldint with ldatm='ck2004'.  Try changing atm to a table that covers a sufficient range of values.  If necessary, set atm to 'blackbody' and/or ld_mode to 'manual' (in which case coefficients will need to be explicitly provided via ld_coeffs). Enable 'warning' logger to see out-of-bound arrays.


In this case, it is necessary to fallback on blackbody atmospheres and manually provide the limb-darkening function and coefficients:

In [9]:
b.set_value('atm', component='primary', value='blackbody')

Fri, 20 Nov 2020 09:54 BUNDLE       WARNING ld_mode='interp' not supported by atm='blackbody'.  Either change atm@primary@phoebe01 or ld_mode@primary@lc01.  If not addressed, this warning will continue to be raised and will throw an error at run_compute.


In [10]:
b.set_value('ld_mode', component='primary', value='manual')

The default values of the `ld_func` and `ld_coeffs` should be changed to appropriate values for a given system, but should still compute without errors.

In [11]:
print(b.filter(qualifier=['ld_func', 'ld_coeffs']))

ParameterSet: 2 parameters
     ld_func@primary@lc01@dataset: logarithmic
   ld_coeffs@primary@lc01@dataset: [0.5 0.5]


In [12]:
b.run_compute()

100%|██████████| 101/101 [00:01<00:00, 58.52it/s]


<ParameterSet: 3 parameters | qualifiers: comments, fluxes, times>